In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
pip -q install langchain langchain-community langchain_groq huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


### USING PICKLE FILE FOR CHAT HISTORY MANAGEMENT

In [ ]:
model_name ='llama3-8b-8192'
groq_key ='key'

In [114]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain, RetrievalQA, ConversationChain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
import pickle

In [ ]:
llm = ChatGroq(model_name=model_name, groq_api_key=groq_key, temperature=0.5, max_tokens=256)

In [ ]:
session_id = 'sundeep_0'
username = 'sun00009'

In [94]:
store = {}

In [95]:
def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]

In [96]:
template = [
        (
            "system",
            "You're a helpful converastion chatbot. Keep your response to limit of 100 words only.",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]

In [97]:
prompt = ChatPromptTemplate.from_messages(template)

In [98]:
model_llm = prompt | llm

In [99]:
conversation = RunnableWithMessageHistory(
    runnable=model_llm,
    get_session_history=get_session_history,
    input_messages_key='question',
    history_messages_key='history',
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name=username,
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name=session_id,
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],)

In [100]:
conversation.invoke({'question':input()},
                    config={"configurable": {"user_id": username, 'conversation_id': session_id}}).content

"Hello! It seems like you're not saying anything. Is there something on your mind that you'd like to talk about or ask? I'm here to help with any questions or concerns you may have. I can also suggest some topics to discuss if you're feeling stuck. Just let me know how I can assist you!"

In [101]:
conversation.invoke({'question':input()},
                    config={"configurable": {"user_id": username, 'conversation_id': session_id}}).content

Hi


"Hi! It's nice to meet you. Is there something on your mind that you'd like to talk about or ask? I'm here to listen and help if I can."

In [102]:
conversation.invoke({'question':input()},
                    config={"configurable": {"user_id": username, 'conversation_id': session_id}}).content

Hi


"Hi again! It seems like we're just exchanging hellos. Is there something specific you'd like to talk about or ask? I'm here to listen and help if I can."

In [113]:
store[username,session_id]

InMemoryChatMessageHistory(messages=[HumanMessage(content=''), AIMessage(content="Hello! It seems like you're not saying anything. Is there something on your mind that you'd like to talk about or ask? I'm here to help with any questions or concerns you may have. I can also suggest some topics to discuss if you're feeling stuck. Just let me know how I can assist you!", response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 37, 'total_tokens': 103, 'completion_time': 0.05191208, 'prompt_time': 0.008296921, 'queue_time': None, 'total_time': 0.060209001}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-27df7441-de81-4533-8f04-834a1550f33c-0', usage_metadata={'input_tokens': 37, 'output_tokens': 66, 'total_tokens': 103}), HumanMessage(content='Hi'), AIMessage(content="Hi! It's nice to meet you. Is there something on your mind that you'd like to talk about or ask? I'm here to listen and help if I

In [118]:
# with open('chat_history.pkl', 'wb') as f:
#     pickle.dump(store, f)

# with open('chat_history.pkl', 'rb') as f:
#     store = pickle.load(f)